# Ensemble Post Processing

Builds monthlies, rootzone soil moisture, extractions

Do not rerun in same folder, it will overwrite existing postprocessed data

In [56]:
from awrams.visualisation import results
from awrams.utils import extents
from awrams.utils import datetools as dt
from awrams.utils import mapping_types as mt
import numpy as np
from matplotlib import pyplot as plt
import os
from awrams.utils.io import data_mapping as dm
import pandas as pd

In [2]:
ENSEMBLE_BASE_PATH = '/data/cwd_awra_data/awra_test_outputs/SydneyWater/'

In [3]:
# Load the Scheduled Run results as a reference - these are the 'actuals' using AWAP data

ref_res = results.load_results('/data/cwd_awra_data/awra_test_outputs/Scheduled_v5_awraprod1/')

In [4]:
# Make variable names consistent between old and new style

from awrams.utils import invert_dict

VARS_OF_INTEREST = ['s0','ss','sm','e0','qtot','etot']
OP_ENS_VAR_MAPPING = dict([(k+'_avg',k) for k in VARS_OF_INTEREST])
ENS_OP_VAR_MAPPING = invert_dict(OP_ENS_VAR_MAPPING)
#NC_VAR_MAPPING = dict([(k,ref_res.variables[ENS_OP_VAR_MAPPING[k]].reader.sfm.mapped_var.variable)\
#                       for k in VARS_OF_INTEREST])

In [5]:
from awrams.utils.gis import ShapefileDB
sw_sf = ShapefileDB('/data/cdc_cwd_wra/awra/afrost/SydneyWater/Area_Operations_region.shp')
sw_extent = sw_sf.get_extent_by_field('ID',1,ref_res.extent)

In [6]:
# The extent object defined by the Sydney Water shapefile

sw_extent

origin: -33.0,149.95, shape: (37, 29), cell_size: 0.05

In [7]:
# Get soil moisture conversion tables

from awrams.utils.nodegraph import graph
from awrams.models import awral
imap = awral.get_default_mapping()

def get_pct_full(actual,ref_grid):
    return actual/ref_grid

cs = mt.gen_coordset(dt.dates('1 jan 2000'),sw_extent) # Arbitrary time coords, only need spatial data

smmap = graph.get_input_tree(['s0max','ssmax','sdmax'],imap.mapping)
smexe = graph.ExecutionGraph(smmap)
smdata = smexe.get_data(cs)
smdata['smmax'] = smdata['s0max'] + smdata['ssmax']

In [8]:
allfcast = os.listdir(ENSEMBLE_BASE_PATH)
allfcast = [k for k in allfcast if k.startswith('fcast')]

In [9]:
all_ens = ['e'+str(i).zfill(2) for i in range(1,12)]
all_ens

['e01', 'e02', 'e03', 'e04', 'e05', 'e06', 'e07', 'e08', 'e09', 'e10', 'e11']

# Create rootzone soil moisture

In [ ]:
assert(False) # Comment out to run
for cursim in allfcast:
    print(cursim)
    eres = results.EnsembleResults(os.path.join(ENSEMBLE_BASE_PATH,cursim))
    eres_cs = mt.gen_coordset(eres.period,eres.extent)
    mapped_smvar = mt.MappedVariable(sm_var,eres_cs,np.float32)
    for e_str in all_ens:
        e_path = os.path.join(ENSEMBLE_BASE_PATH,cursim,e_str)
        print(e_path)
        s0_data = eres.results[e_str].variables.s0.get_data(eres.period,eres.extent) 
        ss_data = eres.results[e_str].variables.ss.get_data(eres.period,eres.extent)
        sm_data = s0_data + ss_data
        sm_data[:,eres.extent.mask] = -999.0

        sm_sfm = dm.SplitFileManager(e_path,mapped_smvar)
        sm_sfm.create_files(dm.AnnualSplitSchema)

        sm_sfm.set_by_coords(eres_cs,sm_data)

        sm_sfm.close_all()

In [10]:
from awrams.utils import ts

# Convert to monthly

In [ ]:



assert(False) # Comment out to run

for cursim in allfcast:
    print(cursim)
    eres = results.EnsembleResults(os.path.join(ENSEMBLE_BASE_PATH,cursim))
    eres_cs = mt.gen_coordset(eres.period,eres.extent)
    
    mres_idx = ts.processing.build_resample_index(eres.period,'m')
    mperiod = dt.resample_dti(eres.period,'m')
    
    for e_str in all_ens:
        
        moutpath = os.path.join(ENSEMBLE_BASE_PATH,'monthly',cursim,e_str)
        

        
            
            print(e_str,v)
            
            mtv = eres.results['e01'].variables[v].reader.sfm.mapped_var.variable
            mapped_v = mt.MappedVariable(mtv,mt.gen_coordset(mperiod,eres.extent),np.float32)
            
            out_sfm = dm.SplitFileManager(moutpath,mapped_v)
            out_sfm.create_files(dm.FlatFileSchema,clobber=True)
            write_cs = out_sfm.get_coords()
            
            d = eres.results[e_str].variables[v].get_data(eres.period,eres.extent)
            if v.startswith('s'):
                rdata = [d[s].mean(axis=0) for s in mres_idx]
            else:
                rdata = [d[s].sum(axis=0) for s in mres_idx]
            rdata = np.array(rdata)
            rdata[:,eres.extent.mask] = -999.0
            
            out_sfm.set_by_coords(write_cs,rdata)
            out_sfm.close_all()
            
            
            
            
            


In [52]:
def agg_volume_pct(in_data,extent,soil_buckets):
    '''
    Return a percentage full aggregated series
    '''
    aweights = (extent.areas / extent.area)
    maxfull = np.nansum(soil_buckets * aweights)
    pct_full = np.nansum(in_data * aweights,axis=(1,2))/maxfull
    return pct_full

In [61]:
def agg_weighted_mean(in_data,extent):
    '''
    Return an area weighted mean aggregation
    '''
    aweights = (extent.areas / extent.area)
    return np.nansum(in_data*aweights,axis=(1,2))

# Extractions

In [62]:
for cursim in allfcast:
    eres = results.EnsembleResults(os.path.join(ENSEMBLE_BASE_PATH,cursim))
    eres_cs = mt.gen_coordset(eres.period,eres.extent)
    
    outpath = os.path.join(ENSEMBLE_BASE_PATH,'extractions',cursim)
    
    print(outpath)
    
    os.makedirs(outpath,exist_ok=True)
    
    
    
    for v in VARS_OF_INTEREST:
        agg_df = pd.DataFrame(index=eres.period)
        pct_df = pd.DataFrame(index=eres.period)
        print(v)            
        v_data = eres.get_ens_results(eres.period,sw_extent,v)

        for e_str in all_ens:

            agg_df[e_str] = agg_weighted_mean(v_data[e_str],sw_extent)

            if v.startswith('s'):
                pct_df[e_str] = agg_volume_pct(v_data[e_str],sw_extent,smdata[v+'max'])

        agg_df.to_csv(os.path.join(outpath,v+'_agg.csv'))

        if v.startswith('s'):
            pct_df.to_csv(os.path.join(outpath,v+'_pct.csv'))
                    
            
            

/data/cwd_awra_data/awra_test_outputs/SydneyWater/extractions/fcast20020501
s0
ss
sm
e0
qtot
etot
/data/cwd_awra_data/awra_test_outputs/SydneyWater/extractions/fcast20101001
s0
ss
sm
e0
qtot
etot
/data/cwd_awra_data/awra_test_outputs/SydneyWater/extractions/fcast20050501
s0
ss
sm
e0
qtot
etot
/data/cwd_awra_data/awra_test_outputs/SydneyWater/extractions/fcast20081001
s0
ss
sm
e0
qtot
etot
/data/cwd_awra_data/awra_test_outputs/SydneyWater/extractions/fcast20011001
s0
ss
sm
e0
qtot
etot
/data/cwd_awra_data/awra_test_outputs/SydneyWater/extractions/fcast20111001
s0
ss
sm
e0
qtot
etot
/data/cwd_awra_data/awra_test_outputs/SydneyWater/extractions/fcast20040501
s0
ss
sm
e0
qtot
etot
/data/cwd_awra_data/awra_test_outputs/SydneyWater/extractions/fcast20030501
s0
ss
sm
e0
qtot
etot
/data/cwd_awra_data/awra_test_outputs/SydneyWater/extractions/fcast20001001
s0
ss
sm
e0
qtot
etot
/data/cwd_awra_data/awra_test_outputs/SydneyWater/extractions/fcast20091001
s0
ss
sm
e0
qtot
etot
/data/cwd_awra_data/

In [65]:
# Load results for processed data (ie soil moisture percentiles/rootzone soil moisture)

ref_processed_res = results.load_results('/data/cwd_awra_data/awra_test_outputs/Scheduled_v5_awraprod1/processed/values/day/')

In [70]:
ACTUAL_PERIOD = dt.dates('1990 - july 2017')

In [71]:
ENS_OP_VAR_MAPPING[v]

's0_avg'

In [72]:
# Extract actuals

outpath = os.path.join(ENSEMBLE_BASE_PATH,'extractions/actuals')
    
print(outpath)

os.makedirs(outpath,exist_ok=True)

actuals_df = pd.DataFrame(index=ACTUAL_PERIOD)

for v in VARS_OF_INTEREST:
    print(v)
    
    if v == 'sm':
        actual_res = ref_processed_res
    else:
        actual_res = ref_res
        
    v_actual = ENS_OP_VAR_MAPPING[v]
    
    v_data = actual_res.variables[v_actual].get_data(ACTUAL_PERIOD,sw_extent)

    for e_str in all_ens:

        actuals_df[v] = agg_weighted_mean(v_data,sw_extent)

        if v.startswith('s'):
            actuals_df[v+'_pct'] = agg_volume_pct(v_data,sw_extent,smdata[v+'max'])

actuals_df.to_csv(os.path.join(outpath,'awap.csv'))               

/data/cwd_awra_data/awra_test_outputs/SydneyWater/extractions/actuals
s0
ss
sm
e0
qtot
etot


In [ ]:
pctiles = [0,5,20,25,50,75,80,95,100]
eda_pct = np.percentile(list(edata_agg.values()),pctiles,axis=0)

In [ ]:
ref_data = ref_res.variables['ss_avg'].get_data(eres.period,sw_extent)
ref_agg = agg_volume_pct(ref_data,smdata['ssmax'])

In [ ]:
df = pd.DataFrame(index=period)
for i,k in enumerate(pctiles):
    df['%spct' % k] = eda_pct[i]
df['actual'] = ref_agg

In [ ]:
import matplotlib
matplotlib.style.use('ggplot')
matplotlib.rcParams['figure.figsize'] = [14,6]
from matplotlib import pyplot as plt

In [ ]:
sm_layer_name = 'Rootzone Soil Moisture'

In [ ]:

df[['5pct','50pct','95pct','actual']].plot(style=dict(actual='.')).legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title('%s (catchment area) - forecast vs AWAP' % sm_layer_name)
plt.ylabel('proportion full')


In [ ]:
def pctile_series(ref,comp):
    
    out = np.empty_like(comp)
    for i in range(len(comp)):
        out[i] = stats.percentileofscore(ref[:,i],comp[i])
    return out

In [ ]:
df[['actual','5pct','50pct','95pct']].plot(style=dict(actual='.'))
plt.title('%s (catchment area) - forecast vs AWAP' % sm_layer_name)
plt.ylabel('mm')

In [ ]:
from awrams.utils.nodegraph import graph
from awrams.models import awral
imap = awral.get_default_mapping()

def get_pct_full(actual,ref_grid):
    return actual/ref_grid

smmap = graph.get_input_tree(['s0max','ssmax','sdmax'],imap.mapping)
smexe = graph.ExecutionGraph(smmap)
smdata = smexe.get_data(cs)

In [ ]:
smmap = graph.get_input_tree(['s0max','ssmax','sdmax'],imap.mapping)
smexe = graph.ExecutionGraph(smmap)
smdata = smexe.get_data(cs)

In [ ]:
smdata['ssmax'].shape

In [ ]:
imap.mapping.s0max

In [ ]:
e01_sfull = get_pct_full(ss_ens['e01'],smdata['ssmax'])

In [ ]:
np.nanmin(e01_sfull)

In [ ]:
plt.imshow(e01_sfull[0])

In [ ]:
d = get_ens_results(eres,period[0:180],extent,'s0')

In [ ]:
dl = list(d.values())

In [ ]:
pctiles = np.percentile(dl,[0,5,20,25,50,75,80,95,100],axis=0)

In [ ]:
mpc = np.ma.MaskedArray(pctiles)

In [ ]:
mpc.mask = True
mpc.mask[...] = False
mpc.mask[np.isnan(pctiles)] = True

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
pctiles.shape

In [ ]:
maxval=np.nanmax(pctiles)

In [ ]:
pctiles.shape

In [ ]:
im = plt.imshow(pctiles[-1][0]-pctiles[0][0])
plt.colorbar(im)

In [ ]:
im = plt.imshow(pctiles[-1][-1]-pctiles[0][-1])
plt.colorbar(im)

In [ ]:
im = plt.imshow(pctiles[4][0],vmin=0.,vmax=maxval)
plt.colorbar(im)

In [ ]:
im = plt.imshow(pctiles[-1][0],vmin=0.,vmax=maxval)
plt.colorbar(im)

In [ ]:
np.nanmin(pctiles[-1,90] - pctiles[0,90])

In [ ]:
ref_data = ref_res.variables.ss_avg.get_data(period,extent)

In [ ]:
ss_ens = get_ens_results(eres,period,extent,'ss')

In [ ]:
a = np.random.normal(size=(3,3,5))
a

In [ ]:
def cell_flatten(x):
    return x.reshape((x.shape[0],x.shape[1]*x.shape[2]))
    

In [ ]:
def naive_corr(x,y):
    output = np.empty(x.shape[1:])
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            output[i,j] = np.corrcoef(x[:,i,j],y[:,i,j])[0,1]
    return output

In [ ]:
np.corrcoef(ref_data[:,52,52],ss_ens['e01'][:,52,52])

In [ ]:
np.corrcoef?

In [ ]:
ref_data.shape[1:]

In [ ]:
ccmap = {}
for k,v in ss_ens.items():
    print(k)
    ccmap[k] = naive_corr(ref_data,v)

In [ ]:
im = plt.imshow(ccmap,vmin=-1,vmax=1)
plt.colorbar(im)

In [ ]:
k = 'e11'
im = plt.imshow(ccmap[k],vmin=-1,vmax=1)
plt.colorbar(im)
plt.title(k)

In [ ]:
ccmean = np.nanmean(list(ccmap.values()),axis=(0))

In [ ]:
plt.imshow(ccmean,vmin=-1,vmax=1)
plt.colorbar(im)
plt.title(k)

In [ ]:
cell_flatten(ref_data).shape

In [ ]:
ccmap.shape

In [ ]:
offset_med = ref_res.variables.ss_avg.data - mpc[4,30]

In [ ]:
offset = ref_res.variables.ss_avg.data - np.mean(dl,axis=0)[30]

In [ ]:
im = plt.imshow(offset_med)
plt.colorbar(im)

In [ ]:
im = plt.imshow(offset)
plt.colorbar(im)

In [ ]:
ref_res.plot_spatial('ss_avg',period[30:31],extent)

In [ ]:
ref_res = results.load_results('/data/cwd_awra_data/awra_test_outputs/Scheduled_v5_awraprod1/')

In [ ]:
ref_states = results.load_results('/data/cwd_awra_data/awra_test_outputs/Scheduled_v5_awraprod1/states/')

In [ ]:
sd_dr = ref_states.variables.sd_dr.get_data(dt.dates('apr 30 2010'),res1.extent)

In [ ]:
sd_sr = ref_states.variables.sd_sr.get_data(dt.dates('apr 30 2010'),res1.extent)

In [ ]:
sd_run = ref_res.variables.sd_avg.get_data(dt.dates('apr 30 2010'),res1.extent)

In [ ]:
res1 = results.load_results('/data/cwd_awra_data/awra_test_outputs/SydneyWater/fcast20100501/e03/')
#res2 = results.load_results('../simulation/notebooks/SydneyWater/e02/')
#res3 = results.load_results('../simulation/notebooks/SydneyWater/e03/')
#res4 = results.load_results('../simulation/notebooks/SydneyWater/e04/')

In [ ]:
res = res1

In [ ]:
res1.period

In [ ]:
res1.plot_spatial(None,res1.period[30:31])

In [ ]:
ref_res.plot_spatial(None,res1.period[30:31],res1.extent)